In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

In [2]:
hit = pd.read_csv("Hitters.csv")
df = hit.copy()
df = df.dropna()
dms = pd.get_dummies(df[['League', 'Division', 'NewLeague']])
y = df["Salary"]
X_ = df.drop(['Salary', 'League', 'Division', 'NewLeague'], axis=1).astype('float64')
X = pd.concat([X_, dms[['League_N', 'Division_W', 'NewLeague_N']]], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.25, 
                                                    random_state=42)

In [ ]:
#%pip install xgboost

   ---------------------------------------- 0.0/150.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/150.0 MB ? eta -:--:--
   ---------------------------------------- 0.3/150.0 MB ? eta -:--:--
   ---------------------------------------- 0.5/150.0 MB 1.1 MB/s eta 0:02:14
   ---------------------------------------- 0.8/150.0 MB 1.0 MB/s eta 0:02:23
   ---------------------------------------- 0.8/150.0 MB 1.0 MB/s eta 0:02:23
   ---------------------------------------- 1.0/150.0 MB 967.3 kB/s eta 0:02:34
   ---------------------------------------- 1.3/150.0 MB 944.7 kB/s eta 0:02:38
   ---------------------------------------- 1.3/150.0 MB 944.7 kB/s eta 0:02:38
   ---------------------------------------- 1.6/150.0 MB 942.3 kB/s eta 0:02:38
   ---------------------------------------- 1.8/150.0 MB 940.7 kB/s eta 0:02:38
   ---------------------------------------- 1.8/150.0 MB 940.7 kB/s eta 0:02:38
    --------------------------------------- 2.1/150.0 MB 876.3 kB/s eta 0

In [4]:
import xgboost as xgb
DM_train = xgb.DMatrix(data=X_train, label=y_train)
DM_test = xgb.DMatrix(data=X_test, label=y_test)

In [7]:
from xgboost import XGBRegressor
xgb_model = XGBRegressor().fit(X_train, y_train)

In [8]:
y_pred = xgb_model.predict(X_test)

In [9]:
np.sqrt(mean_squared_error(y_test, y_pred))

366.3863437634965

MODEL TUNING

In [10]:
xgb_grid = {
     'colsample_bytree': [0.4, 0.5,0.6,0.9,1], 
     'n_estimators':[100, 200, 500, 1000],
     'max_depth': [2,3,4,5,6],
     'learning_rate': [0.1, 0.01, 0.5]
}


In [11]:
xgb_model = XGBRegressor()

In [12]:
xgb_cv_model = GridSearchCV(xgb_model,
                           param_grid=xgb_grid,
                           n_jobs=-1,
                           cv=10,
                           verbose=2)

In [13]:
xgb_cv_model.fit(X_train, y_train)

Fitting 10 folds for each of 300 candidates, totalling 3000 fits


GridSearchCV(cv=10,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, feature_weights=None,
                                    gamma=None, grow_policy=None,
                                    importance_type=None,
                                    interaction_constraints=Non...
                                    max_cat_to_onehot=None, max_delta_step=None,
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None,
                                    multi_strategy=None, n_estimators=None,
                                    n_jobs=None, num_parallel_tree=None, ...),
             n_jobs=-1,
             param_grid={'colsample_bytree': [0.4, 0.5, 0.6, 0.9, 1],
                         'learning_rate': [0.1, 0.01, 0.5],
                         'max_depth': [2, 3, 4, 5, 6],
                         'n_estimators': [100, 200, 500, 1000]},
             verbose=2)

In [14]:
xgb_cv_model.best_params_

{'colsample_bytree': 0.4,
 'learning_rate': 0.1,
 'max_depth': 6,
 'n_estimators': 100}

In [20]:
xgb_tuned = XGBRegressor(learning_rate=xgb_cv_model.best_params_['learning_rate'],
                         max_depth=xgb_cv_model.best_params_['max_depth'],
                        n_estimators=xgb_cv_model.best_params_['n_estimators'],
                        colsample_bytree=xgb_cv_model.best_params_['colsample_bytree']).fit(X_train, y_train)

In [21]:
y_pred_tuned = xgb_tuned.predict(X_test)
np.sqrt(mean_squared_error(y_test, y_pred_tuned))

343.6101991802883